# VU Econometics and Data Science: Case Study
```
Author: Jacco Broere
```

### Setup
- Setup config.ini file
- Install necessary packages
- Download and unpack data



In [1]:
# import packages
import pandas as pd
import numpy as np
import configparser
import os

# helper functions
from helpers.helper_functions import transform_data, add_actuals

# sklearn
from sklearn.decomposition import PCA, SparsePCA

In [2]:
# Read config.ini file
config = configparser.ConfigParser()
config.read('src/config.ini')

['src/config.ini']

### Preprocessing

In [3]:
# Read data
train = pd.read_csv(config['PATH']['RAW_TRAIN_DATA'])
test = pd.read_csv(config['PATH']['RAW_TEST_DATA'])
actuals = pd.read_csv(config['PATH']['ACTUALS'])

In [4]:
# preprocess data
train = transform_data(train)
train = add_actuals(train)

In [7]:
train.head(5)

,Gene Description,Gene Accession Number,1,call,2,call.1,3,call.2,4,call.3,...,29,call.33,30,call.34,31,call.35,32,call.36,33,call.37
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-214,A,-139,A,-76,A,-135,A,...,15,A,-318,A,-32,A,-124,A,-135,A
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-153,A,-73,A,-49,A,-114,A,...,-114,A,-192,A,-49,A,-79,A,-186,A
2,AFFX-BioB-3_at (endogenous control),AFFX-BioB-3_at,-58,A,-1,A,-307,A,265,A,...,2,A,-95,A,49,A,-37,A,-70,A
3,AFFX-BioC-5_at (endogenous control),AFFX-BioC-5_at,88,A,283,A,309,A,12,A,...,193,A,312,A,230,P,330,A,337,A
4,AFFX-BioC-3_at (endogenous control),AFFX-BioC-3_at,-295,A,-264,A,-376,A,-419,A,...,-51,A,-139,A,-367,A,-188,A,-407,A
